In [1]:
from project_fraud.lib import drop_many_missing_values
from project_fraud.feature_engineering import clean_mail, make_day_feature, make_hour_feature, string_card, credit_cards, dist_from_mean, dist_from_median_rel


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import seaborn as sns


from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LassoCV
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression

In [2]:
# transform_raw_data

In [3]:
def transform_raw_data():
    data = drop_many_missing_values()
    data['P_emaildomain_bin'], data['P_emaildomain_suffix'] = clean_mail(data)
    data['weekday'] = make_day_feature(data, offset=0.58)
    data['hours'] = make_hour_feature(data)
    data['cardID'] = data.apply(lambda row: string_card(row), axis=1)
    data = data.merge(credit_cards(data), how='left', on="cardID")
    data['dist_mean'] = data.apply(lambda row: dist_from_mean(row, 'mean'), axis=1)
    data['dist_median'] = data.apply(lambda row: dist_from_mean(row, 'median'), axis=1)
    data['dist_mean_rel'] = data.apply(lambda row: dist_from_median_rel(row, 'mean'), axis=1)
    data['dist_median_rel'] = data.apply(lambda row: dist_from_median_rel(row, 'median'), axis=1)
    return data

In [ ]:
data_transformed = transform_raw_data()

In [ ]:
data_transformed.head()

In [39]:
data_transformed.P_emaildomain_suffix

0         nan
1         com
2         com
3         com
4         com
         ... 
590535    nan
590536    com
590537    com
590538    com
590539    com
Name: P_emaildomain_suffix, Length: 590540, dtype: object

In [36]:
type(data_transformed.P_emaildomain_suffix[590535])

str

In [40]:
def clean_mail_suffix(P_emaildomain_suffix):
    if P_emaildomain_suffix == 'nan':
        return np.nan
    else:
        return P_emaildomain_suffix

In [42]:
data_transformed['P_emaildomain_suffix'] = data_transformed['P_emaildomain_suffix'].apply(lambda x: clean_mail_suffix(x))

In [44]:
# Pipeline

In [3]:
X = data_transformed[['TransactionID','P_emaildomain_suffix','P_emaildomain_bin',
'card1','card2','addr1','TransactionAmt','card5','D15','C13','D2','D10','D4','weekday','hours',\
           'dist_mean', 'dist_median','dist_mean_rel','dist_median_rel']]
y = data_transformed['isFraud']

NameError: name 'data_transformed' is not defined

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.00017, random_state=1)

NameError: name 'X' is not defined

In [5]:
X_val.shape

NameError: name 'X_val' is not defined

In [49]:
type(X_val['P_emaildomain_suffix'][390381])

float

In [50]:
X_val.to_csv('val_data.csv', index =False, na_rep='nan')

In [51]:
n = (X.dtypes != 'object')
num_cols = list(n[n].index)
medium_missing_num_cols = []
low_missing_num_cols =[]
for i in num_cols:
    percentage = data_transformed[i].isnull().sum() * 100 / len(data_transformed[i])
    if percentage < 15:
        low_missing_num_cols.append(i)
    elif percentage >= 15 and percentage <= 60:
        medium_missing_num_cols.append(i)

In [52]:
# trainer.py

num_transformer_low = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
num_transformer_medium = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('one_hot', OneHotEncoder())
])
    
preprocessor = ColumnTransformer([
    ('low_num_imputer',num_transformer_low, low_missing_num_cols),
    ('medium_num_imputer', num_transformer_medium, medium_missing_num_cols),
    ('cat_transformer', cat_pipeline, ['P_emaildomain_suffix','P_emaildomain_bin','weekday','hours'])],
    remainder='drop')

In [53]:
final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('log_reg', LogisticRegression(class_weight ='balanced'))])

In [54]:
final_pipe.fit(X_train, y_train)

/Users/gillesfaltz/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('preprocessing', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('low_num_imputer', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
 ...penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False))])

In [55]:
final_pipe.predict(X_val)

array([0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1])

In [56]:
import pickle

# Export pipeline as pickle file
with open("pipeline_8.pkl", "wb") as file:
    pickle.dump(final_pipe, file)